<a href="https://colab.research.google.com/github/zinia00/EDGE-ASSIGNMENT/blob/main/IMDB_Rate_Prediction_(Linear_Regression).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('imdb.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6178 entries, 0 to 6177
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         6178 non-null   object 
 1   Date         6178 non-null   int64  
 2   Rate         6178 non-null   object 
 3   Votes        6178 non-null   object 
 4   Genre        6178 non-null   object 
 5   Duration     5877 non-null   float64
 6   Type         6178 non-null   object 
 7   Certificate  5728 non-null   object 
 8   Episodes     6178 non-null   object 
 9   Nudity       4719 non-null   object 
 10  Violence     5504 non-null   object 
 11  Profanity    5520 non-null   object 
 12  Alcohol      5407 non-null   object 
 13  Frightening  5320 non-null   object 
dtypes: float64(1), int64(1), object(12)
memory usage: 675.8+ KB


In [4]:
df.head()

,Name,Date,Rate,Votes,Genre,Duration,Type,Certificate,Episodes,Nudity,Violence,Profanity,Alcohol,Frightening
0,No Time to Die,2021,7.6,"107,163","Action, Adventure, Thriller",163.0,Film,PG-13,-,Mild,Moderate,Mild,Mild,Moderate
1,The Guilty,2021,6.3,"64,375","Crime, Drama, Thriller",90.0,Film,R,-,NaN,NaN,Severe,NaN,Moderate
2,The Many Saints of Newark,2021,6.4,"27,145","Crime, Drama",120.0,Film,R,-,Moderate,Severe,Severe,Moderate,Moderate
3,Venom: Let There Be Carnage,2021,6.4,"30,443","Action, Adventure, Sci-Fi",97.0,Film,PG-13,-,NaN,Moderate,Moderate,Mild,Moderate
4,Dune,2021,8.3,"84,636","Action, Adventure, Drama",155.0,Film,PG-13,-,NaN,Moderate,NaN,Mild,Moderate


In [5]:
df.drop(['Name', 'Votes'], axis=1, inplace=True)

In [6]:
df.drop(df[df.Rate=='No Rate'].index, inplace=True)
df.drop(df[(df.Certificate == 'Not Rated') | (df.Certificate == 'Unrated')].index, inplace=True)

cols_include_none = ['Duration', 'Certificate', 'Nudity', 'Violence', 'Profanity', 'Alcohol', 'Frightening']

for col in cols_include_none:
    df.drop(df[(df[col]=='None') | (df[col]=='No Rate')].index, inplace=True)

In [7]:
df['Rate'] = pd.to_numeric(df['Rate'], downcast='float')
df['Duration'] = pd.to_numeric(df['Duration'], downcast='unsigned')

df.loc[df.Episodes == '-', 'Episodes'] = 1
df['Episodes'] = pd.to_numeric(df['Episodes'], downcast='unsigned')

In [8]:
df.Genre = df.Genre.str.replace(' ', '')
genre_cols = df.Genre.str.get_dummies(sep=',')

df.drop('Genre', axis=1, inplace=True)

df = pd.concat([df, genre_cols], axis=1)

In [9]:
categorical_cols = ['Type','Certificate', 'Nudity', 'Violence', 'Profanity', 'Alcohol', 'Frightening']

df = pd.get_dummies(df, columns=categorical_cols)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5012 entries, 0 to 6177
Data columns (total 68 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Date                  5012 non-null   int64  
 1   Rate                  5012 non-null   float32
 2   Duration              4947 non-null   float64
 3   Episodes              5012 non-null   uint16 
 4   Action                5012 non-null   int64  
 5   Adventure             5012 non-null   int64  
 6   Animation             5012 non-null   int64  
 7   Biography             5012 non-null   int64  
 8   Comedy                5012 non-null   int64  
 9   Crime                 5012 non-null   int64  
 10  Documentary           5012 non-null   int64  
 11  Drama                 5012 non-null   int64  
 12  Family                5012 non-null   int64  
 13  Fantasy               5012 non-null   int64  
 14  Film-Noir             5012 non-null   int64  
 15  Game-Show             5012

In [11]:
X = df.drop(['Rate'], axis=1)
y = df['Rate'].values

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [15]:
# Check if NaNs exist in the dataset
print("Number of NaN values in X_train:", np.isnan(X_train).sum())
print("Number of NaN values in y_train:", np.isnan(y_train).sum())

Number of NaN values in X_train: Date                     0
Duration                51
Episodes                 0
Action                   0
Adventure                0
                        ..
Alcohol_Moderate         0
Alcohol_Severe           0
Frightening_Mild         0
Frightening_Moderate     0
Frightening_Severe       0
Length: 67, dtype: int64
Number of NaN values in y_train: 0


In [19]:
from sklearn.impute import SimpleImputer


# Fill NaNs in "Duration" with mean
imputer = SimpleImputer(strategy="mean")
X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(imputer.fit_transform(X_test), columns=X_test.columns)


# Train the model
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(X_train, y_train)


LinearRegression()

In [20]:
predictions = regressor.predict(X_test)

In [21]:
predictions[2]

6.235293861060654

In [22]:
y_test

array([7.1, 7.6, 6.4, ..., 6.7, 7.2, 5.9], dtype=float32)

In [23]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, predictions))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, predictions))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

Mean Absolute Error: 0.6329102918222567
Mean Squared Error: 0.7140539205139339
Root Mean Squared Error: 0.8450171125568605


In [24]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(regressor, X_train, y_train, scoring="neg_mean_squared_error", cv=10)

reg_rmse_scores = np.sqrt(-scores)

print("Scores:", scores)
print("Mean:", scores.mean())
print("Standard deviation:", scores.std())

Scores: [-0.66760834 -0.6900425  -0.82870934 -0.61892581 -0.666141   -0.73186305
 -0.68720203 -0.67017945 -0.72854537 -0.63312012]
Mean: -0.69223370009516
Standard deviation: 0.056733438949435694
